In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

dataset=pd.read_csv("Churn_Modelling.csv")

In [2]:
X=dataset.iloc[:,3:13].values
Y=dataset.iloc[:,13].values

In [3]:
from sklearn import preprocessing
le_X_1=preprocessing.LabelEncoder()
le_X_2=preprocessing.LabelEncoder()
X[:,1]=le_X_1.fit_transform(X[:,1])
X[:,2]=le_X_2.fit_transform(X[:,2])

In [4]:
#for dummy encoding only on dummy with three values and not for two
from sklearn.preprocessing import OneHotEncoder
ohe_1=OneHotEncoder(categorical_features = [1])
X=ohe_1.fit_transform(X).toarray()
X=X[:,1:]

In [5]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.20,random_state=0)
print(X_train)

[[0.0000000e+00 1.0000000e+00 6.6700000e+02 ... 1.0000000e+00
  0.0000000e+00 1.6383064e+05]
 [1.0000000e+00 0.0000000e+00 4.2700000e+02 ... 1.0000000e+00
  1.0000000e+00 5.7098000e+04]
 [0.0000000e+00 0.0000000e+00 5.3500000e+02 ... 1.0000000e+00
  0.0000000e+00 1.8563076e+05]
 ...
 [0.0000000e+00 0.0000000e+00 7.3800000e+02 ... 1.0000000e+00
  0.0000000e+00 1.8142987e+05]
 [0.0000000e+00 1.0000000e+00 5.9000000e+02 ... 1.0000000e+00
  1.0000000e+00 1.4875016e+05]
 [1.0000000e+00 0.0000000e+00 6.2300000e+02 ... 1.0000000e+00
  0.0000000e+00 1.1885526e+05]]


In [6]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [7]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.layers import Dropout

C:\Users\Sahil\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [8]:
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(output_dim=6,init = 'uniform', activation = 'relu', input_dim=11))
    classifier.add(Dropout(p=0.1))
    classifier.add(Dense(output_dim=6,init = 'uniform', activation = 'relu'))
    classifier.add(Dropout(p=0.1))
    classifier.add(Dense(output_dim=1,init = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier


In [ ]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size=25,nb_epoch = 5)
accuracies=cross_val_score(estimator = classifier,X=X_train,y=Y_train,cv=10,n_jobs=-1)

In [ ]:
#tuning the ann
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.grid_searchcv import GridSearchCV
from keras.layers import Dropout

def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(output_dim=6,init = 'uniform', activation = 'relu', input_dim=11))
    classifier.add(Dropout(p=0.1))
    classifier.add(Dense(output_dim=6,init = 'uniform', activation = 'relu'))
    classifier.add(Dropout(p=0.1))
    classifier.add(Dense(output_dim=1,init = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier


In [ ]:
classifier = KerasClassifier(build_fn=build_classifier)
parameters = {'batch_size':[25,32],
             'nb_epoch':[100,500],
             'optimizer':['adam','rmsdrop']}
grid_search=GridSearchCV(estimator= classifier,param_grid=parameters,scoring='accuracy',cv='10')
grid_search=grid_search.fit(X_train,Y_train)

best_parameters= grid_search.best_params_
best_accuracy= grid_search.best_score_